# Data Experiment Notebook

### 1. Package Setup

In [2]:
import os
import re
import time
import string
import mlflow
import logging
import dagshub
import pandas as pd
import mlflow.sklearn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)# Save a smaller sample for quicker runs
df.head()

,review,sentiment
836,"I know it's not original, but what the hey? Wh...",negative
726,Bottom has been my favourite sitcom ever since...,positive
972,What a good movie! At last a picture revealing...,positive
906,"Stealing is a crime, and these guys, Kenny Yak...",negative
554,"As hard as it is for me to believe, with all o...",negative


### 2. Data Preprocessing

In [4]:
# lemmatization is a function that reduces words to their base or root form.
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()# Initialize the lemmatizer
    text = text.split()# Split the text into words
    text = [lemmatizer.lemmatize(word) for word in text]# Lemmatize each word
    return ' '.join(text)# Join the words back into a single string

#stopwords are common words that are often removed from text data during preprocessing.
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))# Get the set of English stop words
    # converting each word to lowercase and keeping it only if not stop word
    text = [word for word in text.split() if word.lower() not in stop_words]
    return ' '.join(text)# Join the words back into a single string

def removing_numbers(text):
    text = ''.join([char for char in text if not char.isdigit()])# Remove digits from the text
    return text # Return the cleaned text

def lower_case(text):
    text = text.split()# Split the text into words
    text = [word.lower() for word in text]# Convert each word to lowercase
    return " ".join(text)# Join the words back into a single string

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)# Remove punctuation
    text = text.replace('؛', "")# Remove specific punctuation
    text = re.sub('\s+', ' ', text).strip()# Remove extra spaces
    return text

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')# Pattern to match URLs
    return url_pattern.sub(r'', text)

def normalize_text(df):
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:27: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
<>:27: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
/var/folders/4y/cdgg2mhs3474pdf_x3256qn00000gn/T/ipykernel_41574/83381208.py:27: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
  text = re.sub('\s+', ' ', text).strip()# Remove extra spaces


In [5]:
import nltk
# Download required NLTK resources for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adityamishra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/adityamishra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,review,sentiment
836,know original hey else said it feel unutterabl...,negative
726,bottom favourite sitcom ever since saw t v mov...,positive
972,good movie last picture revealing unknown side...,positive
906,stealing crime guy kenny yakkel corbin bernsen...,negative
554,hard believe awful reality show past year one ...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    259
positive    241
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive', 'negative'])# Filter for positive and negative sentiments
df = df[x]# Keep only the filtered rows

In [8]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df.head()

,review,sentiment
836,know original hey else said it feel unutterabl...,0
726,bottom favourite sitcom ever since saw t v mov...,1
972,good movie last picture revealing unknown side...,1
906,stealing crime guy kenny yakkel corbin bernsen...,0
554,hard believe awful reality show past year one ...,0


In [9]:
df.isnull().sum()# Check for null values

review       0
sentiment    0
dtype: int64

In [10]:
# Initialize the CountVectorizer for text feature extraction
vectorizer = CountVectorizer()
# Transform the review texts into feature vectors
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']  # Target variable

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [12]:
mlflow.set_tracking_uri("https://dagshub.com/aaditya29/MLOps-Complete-Project.mlflow")
dagshub.init(repo_owner='aaditya29',
             repo_name='MLOps-Complete-Project', mlflow=True)# Initialize DagsHub for MLflow tracking

mlflow.set_experiment("Logistic Regression Baseline")# Set the MLflow experiment name

Accessing as aaditya29

Initialized MLflow to track repo "aaditya29/MLOps-Complete-Project"

Repository aaditya29/MLOps-Complete-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/27539df8924e4bd4a3df7ff08cb74912', creation_time=1766918274183, experiment_id='0', last_update_time=1766918274183, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
logging.basicConfig(level=logging.INFO,
                    # Configure logging
                    format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")  # Start an MLflow run

with mlflow.start_run():
    start_time = time.time()  # Record the start time

    try:
        logging.info("Logging preprocessing parameters!")
        # Log the vectorizer type
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        # Increase max_iter to prevent non-convergence issues
        model = LogisticRegression(max_iter=1000)

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(
            f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-12-28 16:42:59,883 - INFO - Starting MLflow run...
2025-12-28 16:43:00,371 - INFO - Logging preprocessing parameters!
2025-12-28 16:43:01,526 - INFO - Initializing Logistic Regression model...
2025-12-28 16:43:01,528 - INFO - Fitting the model...
2025-12-28 16:43:01,618 - INFO - Model training complete.
2025-12-28 16:43:01,618 - INFO - Logging model parameters...
2025-12-28 16:43:01,998 - INFO - Making predictions...
2025-12-28 16:43:02,001 - INFO - Calculating evaluation metrics...
2025-12-28 16:43:02,014 - INFO - Logging evaluation metrics...
2025-12-28 16:43:03,460 - INFO - Saving and logging the model...
2025/12/28 16:43:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-28 16:43:14,910 - INFO - Model training and logging completed in 14.54 seconds.
2025-12-28 16:43:14,912 - INFO - Accuracy: 0.728
2025-12-28 16:43:14,913 - INFO - Precision: 0.7285714285714285
2025-12-28 16:43:14,913 - INFO - Recall: 0.7727272727272727
2025-12-28 1

🏃 View run masked-bee-406 at: https://dagshub.com/aaditya29/MLOps-Complete-Project.mlflow/#/experiments/0/runs/a75e3010239441daa5894c76e5ff0699
🧪 View experiment at: https://dagshub.com/aaditya29/MLOps-Complete-Project.mlflow/#/experiments/0
